<a href="https://colab.research.google.com/github/sahaanirbannew/spacy-custom-NER-bird-names/blob/main/Connecting_to_Dropbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Process:**


1.   Edit a photograph on Adobe Lightroom, export it to a Dropbox folder.
2.   Program takes image from Dropbox folder, posts it with description. 



In [3]:
#!pip3 install dropbox

In [77]:
import pathlib
import pandas as pd
import dropbox
from dropbox.exceptions import AuthError
import tweepy 
import pickle 

In [83]:
def to_log(type_,text):
  if type_ == "i":
    log["info"].append(text)
  if type_ == "e":
    log["error"].append(text)


def connect_to_dropbox():
  try:
    TOKEN = "sl.BOspRcQ35ip02mXPUzE6qGd3om4U5de2Hvfqp9xJujXVHMwhVNxg2g6bxlyUTo7SN10YA1j3HwTwOgOd6hH9hhcT4kf-DM2LREN2pDl-SMMM_LxzHacKMRkPCUy53uzsMWJ4zn9B"
    dbx = dropbox.Dropbox(TOKEN)
    to_log("i","Dropbox connection made.") 
    return dbx 
  except Exception as e:
    to_log("e","Dropbox connection failed.")  

def list_files_in_folder(dbx, dropbox_folder):
  files_ = []
  try: 
    files = dbx.files_list_folder(dropbox_folder).entries
    if len(files)>0: to_log("i","There are files existing in the folder.") 
    for file in files:
      files_.append([file.name,file.path_lower]) 
    return files_
  except Exception as e:
    to_log("e",str(e)) 

def fetch_already_uploaded_image_list():
  file = open("/content/already_uploaded_image_list",'rb')
  bird_list_df = pickle.load(file) 
  return bird_list_df

def create_twitter_app_obj():
  consumer_key = "iPaIdR8GRI59yTJMs0Es0dIBN"
  consumer_secret = "pLadg3UaLeK3yKDujRMChRN3p8hUDBOjBsuOBy8j8ERr4zz1vs"
  access_token = "39085479-AabHt6bmFSbClDfUZuHjModYPAxVlOxHeMA79UyVt"
  access_token_secret = "3IqXDISfqg14wzMNNn2AX4KYG9Wfkltt21QxKasE4YNnG"
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret) 

  try:
    api = tweepy.API(auth) 
    return api
  except Exception as e:
   to_log("e",str(e))
  return 0

def download_files_from_dropbox(old_list):
  files = dbx.files_list_folder("/to_twitter").entries
  for file in files:
    if file.name in files__:
      file_path=file.path_lower
      file_name=file.name
      dbx.files_download_to_file(download_path="/content/"+file_name,path=file_path)

def dump_already_uploaded_image_list(list_of_files):
  filepath = open("/content/already_uploaded_image_list",'wb') 
  pickle.dump(list_of_files, filepath)                     
  filepath.close() 


In [111]:
log = {}
log["info"] = []
log["error"] = [] 
dropbox_folder = "/to_twitter"
dbx = connect_to_dropbox()

already_uploaded_image_list = pd.DataFrame(fetch_already_uploaded_image_list() , columns =['name', 'filepath'])  
present_set_of_images = pd.DataFrame(list_files_in_folder(dbx, dropbox_folder) , columns =['name', 'filepath'])
to_upload = present_set_of_images[~present_set_of_images.name.isin(already_uploaded_image_list.name)]


In [112]:
print(to_upload.iloc[0]["name"])
path = to_upload.iloc[0]["filepath"]
local_path = "/content/"+to_upload.iloc[0]["name"] 

DSC_6582.png


In [113]:
download_status = dbx.files_download_to_file(local_path,path)

ApiError: ignored

In [108]:
twitter = create_twitter_app_obj() 
media = twitter.media_upload(local_path)
tweet_text = "This would be automated soon."
post_result = twitter.update_status(status=tweet_text, media_ids=[media.media_id])

TweepError: ignored